In [1]:
!pip install gensim --upgrade
!pip install keras --upgrade
!pip install pandas --upgrade

  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/fc/a7/0d4490de967a67f68a538cc9cdb259bff971c4b5787f7765dc7c8f118f71/keras-2.15.0-py3-none-any.whl.metadata
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.15.0 which is incompatible.


In [4]:
#setting up environment with the necessary tools and libraries for an NLP project, 

#including data handling, visualization, machine learning, deep learning, and NLP-specific tasks.

# DataFrame
import pandas as pd
import gensim #NLP lib

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#functions that are used for evaluating the performance of a ML model
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer


# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re  #regex
import numpy as np
import os  #interacts with the operating system
from collections import Counter #for counting the frequency of elements in a collection
import logging
import time
import pickle
import itertools

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

SymbolAlreadyExposedError: Symbol Zeros is already exposed as ().

In [5]:
import tensorflow.keras as keras

SymbolAlreadyExposedError: Symbol Zeros is already exposed as ().

In [ ]:
""" download the stopwords dataset provided by the NLTK library 
Once downloaded, you can use these stopwords in your NLP tasks for tasks like text cleaning and analysis"""


nltk.download('stopwords')


"""returns Ture if,download for stopeord dataset was successful & is now available for use in NLTK-based NLP tasks."""

In [ ]:
# DATASET
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"] #list of column names that are expected to be found in a dataset
DATASET_ENCODING = "ISO-8859-1" #specifies the encoding format used for reading a dataset
TRAIN_SIZE = 0.8 #0.8 or 80% of the data will be used for training

# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# WORD2VEC 
#parameters used for configuring a Word2Vec model
W2V_SIZE = 300 #size of the word vectors 
W2V_WINDOW = 7
W2V_EPOCH = 32 #no of iterations (epochs) over the corpus during training
W2V_MIN_COUNT = 10 #Ignores all words with a total frequency lower than this

# KERAS
#parameters related to training a neural network model using Keras
SEQUENCE_LENGTH = 300 #length of input sequences(consist of 300 data points/words in NLP) (data points are the atomic elements that are analyzed, processed)
EPOCHS = 8 #no of training iterations
BATCH_SIZE = 1024 #model to process 1024 samples in each iteration/epoch

# SENTIMENT
#labels representing different sentiment categories.
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7) #threshold values, <0.4=neg, >0.7=pos, btw=neu

# EXPORT
#File names used for exporting ML models and processing tools
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl"

In [ ]:
import pandas as pd

# Define the file path
file_path = '/Users/nupurmehlawat/Downloads/training.1600000.processed.noemoticon.csv'

# Read the CSV file
df = pd.read_csv(file_path, header=None, encoding='ISO-8859-1', names=['target', 'id', 'date', 'flag', 'user', 'text'])

# Display the first few rows(first 5) of the DataFrame
print(df.head())

In [ ]:
print("Dataset size:", len(df)) #print total number of rows in the DataFrame/dataset being read

In [ ]:
df.head(5) # displays first five columns

In [ ]:
#Map target label to String
#0 -> NEGATIVE
#2 -> NEUTRAL
#4 -> POSITIVE

decode_map = {0: "NEGATIVE", 2: "NEUTRAL", 4: "POSITIVE"}
def decode_sentiment(label):  #convert numerical sentiment labels(0, 2, 4) into corresponding sentiment categories
    return decode_map[int(label)]

In [ ]:
%%time
df.target = df.target.apply(lambda x: decode_sentiment(x))

In [ ]:
target_cnt = Counter(df.target)

plt.figure(figsize=(16,8)) #fig width, height
plt.bar(target_cnt.keys(), target_cnt.values()) #x-label, y-label resp.
plt.title("Dataset labels distribuition")

#label "positive" hits 800,000, it means that "positive" words appears 800,000 times in your dataset.

In [ ]:
stop_words = stopwords.words("english") #use NLTK library to retrieve a list of common English stopwords
stemmer = SnowballStemmer("english") #initializes a stemmer using the Snowball stemmer algorithm for English

In [ ]:
#preprocess function is designed to clean and prepare text data for further analysis
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []  #store individual words from the cleaned text
    for token in text.split(): #loop iterates through each word in the cleaned text
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token)) #applies stemmer to word & appends stemmmed word to tokens list
            else:
                tokens.append(token)
    return " ".join(tokens)

In [ ]:
%%time
df.text = df.text.apply(lambda x: preprocess(x))

#code cell is timing how long it takes to clean and preprocess the text data in the df.text column using the preprocess function

In [ ]:
#Split train and test
df_train, df_test = train_test_split(df, test_size=1-TRAIN_SIZE, random_state=42)
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

In [ ]:
%%time
documents = [_text.split() for _text in df_train.text] #documents list of word used for training from text col wrt df_train

In [ ]:
from gensim.models import Word2Vec

# Define your hyperparameters
W2V_SIZE = 100 #each word will be represented as a 100-dimensional vector
W2V_WINDOW = 5 #5 words before and 5 words after
W2V_MIN_COUNT = 1 #how many times a word must come, to be included by the word2vec model

# Initialize the Word2Vec model
w2v_model = Word2Vec(vector_size=W2V_SIZE, 
                    window=W2V_WINDOW, 
                    min_count=W2V_MIN_COUNT, 
                    workers=8)

In [ ]:
#prepares the Word2Vec model to generate word embeddings for the words in the provided documents

w2v_model.build_vocab(documents)

In [ ]:

words = list(w2v_model.wv.key_to_index.keys()) #wv=word_vector & key_to_index is a dictionary that maps words to their corresponding integer indices.
vocab_size = len(words)  #total number of unique words in the Word2Vec model
print("Vocab size:", vocab_size)

In [ ]:
%%time
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

#time taken for the training process
#using a list of documents as training data

In [ ]:
from gensim.models import Word2Vec

if hasattr(w2v_model, 'wv') and hasattr(w2v_model.wv, 'most_similar'):
    similar_words = w2v_model.wv.most_similar("love")
    print(similar_words)
else:
    print("The model does not have the 'most_similar' method.")

In [ ]:
%%time
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.text)

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

In [ ]:
%%time
x_train = pad_sequences(tokenizer.texts_to_sequences(df_train.text), maxlen=SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(df_test.text), maxlen=SEQUENCE_LENGTH)

In [ ]:
#Label Encoder
labels = df_train.target.unique().tolist()
labels.append(NEUTRAL)
labels